In [1]:
from __future__ import print_function
import keras.backend as K
from keras.layers import Input, Convolution1D, MaxPooling1D, Dense, Flatten, Reshape, UpSampling1D, Dropout
from keras.metrics import categorical_accuracy
from keras.models import Model, load_model
from keras.objectives import mean_squared_error

import os
os.chdir('..')

from evolutron.networks import Deconvolution1D, Dedense
from evolutron.engine import DeepTrainer

Using Theano backend.


In [2]:
aa_length = 100
# Input Layer, assumes 'tf' ordering of dimensions, i.e. (batch_size, length, channels)
inp = Input(shape=(aa_length, 20), name='aa_seq')

In [3]:
n_filters = 30
filter_length = 10
n_conv_layers = 3
# Convolutional Layers
convs = [Convolution1D(n_filters, filter_length,
                       init='glorot_uniform',
                       activation='relu',
                       border_mode='same',
                       name='Conv1')(inp)]

for c in range(1, n_conv_layers):
    convs.append(Convolution1D(n_filters, filter_length,
                               init='glorot_uniform',
                               activation='relu',
                               border_mode='same',
                               name='Conv{}'.format(c+1))(convs[-1]))  # maybe add L1 regularizer

In [4]:
# Max-pooling
max_pool = MaxPooling1D(pool_length=aa_length)(convs[-1])

flat = Flatten()(max_pool)

In [5]:
# Fully-Connected encoding layers
n_fc_layers=3
fc_enc = [Dense(n_filters,
                init='glorot_uniform',
                activation='sigmoid',
                name='FCEnc1')(flat)]

for d in range(1, n_fc_layers):
    fc_enc.append(Dense(n_filters,
                        init='glorot_uniform',
                        activation='sigmoid',
                        name='FCEnc{}'.format(d+1))(fc_enc[-1]))

encoded = fc_enc[-1]  # To access if model for encoding needed

In [6]:
# Fully-Connected decoding layers
fc_dec = [Dedense(encoded._keras_history[0],
                  activation='linear',
                  name='FCDec{}'.format(n_fc_layers))(encoded)]

for d in range(n_fc_layers-2, -1, -1):
    fc_dec.append(Dedense(fc_enc[d]._keras_history[0],
                          activation='linear',
                          name='FCDec{}'.format(d+1))(fc_dec[-1]))


In [7]:
# Reshaping and unpooling
unflat = Reshape(max_pool._keras_shape[1:])(fc_dec[-1])
deconvs = [UpSampling1D(length=aa_length, name='Unpooling')(unflat)]

In [8]:
# Deconvolution
for c in range(n_conv_layers-1, 0, -1):
    print(c)
    deconvs.append(Deconvolution1D(convs[c]._keras_history[0],
                                   activation='relu',
                                   name='Deconv' + str(c+1))(deconvs[-1]))  # maybe add L1 regularizer
decoded = Deconvolution1D(convs[0]._keras_history[0], activation='sigmoid', name='Deconv1')(deconvs[-1])

2
1


In [9]:
coder = DeepTrainer(Model(input=inp, output=decoded))

In [10]:
coder.n_params

27050

In [13]:
coder.display_network_info()

Neural Network has 27050 trainable parameters
  #  Name            Shape      Parameters
---  --------------  -------  ------------
  0  aa_seq          100x20              0
  1  Conv1           100x30           6030
  2  Conv2           100x30           9030
  3  Conv3           100x30           9030
  4  maxpooling1d_1  1x30                0
  5  flatten_1       30                  0
  6  FCEnc1          30                930
  7  FCEnc2          30                930
  8  FCEnc3          30                930
  9  FCDec3          30                 30
 10  FCDec2          30                 30
 11  FCDec1          30                 30
 12  reshape_1       1x30                0
 13  Unpooling       100x30              0
 14  Deconv3         100x30             30
 15  Deconv2         100x30             30
 16  Deconv1         100x20             20


In [12]:
print(coder.network.input_shape)
print(coder.network.output_shape)

(None, 100, 20)
(None, 100, 20)
